In [1]:
from resnet18 import ResNet, ResidualBlock, data_loader
import numpy as np
import torch
import torch.nn as nn
import torch.backends
import torch.backends.mps
import torch.mps
from tqdm import tqdm

Setting RESNET-18 device as mps


In [2]:
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Setting TEST device as {device}")

device = torch.device(device)

60000
Setting TEST device as mps


In [12]:
num_classes = 10
num_epochs = 20
# batch_size = 16
learning_rate = 0.01

model = ResNet().to(device)
model.requires_grad_ = True
# model = ResNet()
model.make_backbone(ResidualBlock, [2, 2, 2, 2], [1, 3], num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  

# Train the model
# total_step = len(train_loader)
print(len(train_loader) + len(valid_loader))

import gc
total_step = len(train_loader)
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} ================")
    # n_batch = (n_sample - 1) // batch_size + 1
    # model.train()
    total_sample = 0
    total_loss = total_accuracy = 0
    with tqdm(total=64) as dadyNgoy:
        for i, (images, labels) in enumerate(train_loader): 
            # print(i) 
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs[-1].data, 1)
            loss = criterion(outputs[-1], labels)
            total_accuracy += (predicted == labels).sum().item()
            total_loss += loss
            total_sample += labels.size(0)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            del images, labels, outputs
            torch.mps.empty_cache() if torch.backends.mps.is_available() else torch.cuda.empty_cache()
            gc.collect()

            if (i + 1) % 3 == 0 or i + 1 == 64:
                dadyNgoy.set_postfix({
                            'avg_loss': total_loss / (i + 1),
                            'avg_accuracy': total_accuracy / (i + 1),
                            # 'max_abs_gradient': np.max(abs(model.weight.grad))
                        })
                cur_n_batch = i % 3 + 1
                dadyNgoy.update(cur_n_batch)

    print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss.item()))
    accuracy = 100 * total_accuracy / total_sample
    print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}%')
        
    model.eval()
                
        # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs[-1].data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

938
Epoch 1 ================


844it [01:15, 11.25it/s, avg_loss=tensor(0.1403, device='mps:0', grad_fn=<DivBackward0>), avg_accuracy=61.3]                       


Epoch [1/20], Loss: 0.2261
Epoch 1: Accuracy = 95.71%
Accuracy of the network on the 5000 validation images: 98.13333333333334 %
Epoch 2 ================


844it [01:14, 11.28it/s, avg_loss=tensor(0.0667, device='mps:0', grad_fn=<DivBackward0>), avg_accuracy=62.7]                       


Epoch [2/20], Loss: 0.0085
Epoch 2: Accuracy = 97.91%
Accuracy of the network on the 5000 validation images: 98.76666666666667 %
Epoch 3 ================


...

844it [01:15, 11.13it/s, avg_loss=tensor(0.0161, device='mps:0', grad_fn=<DivBackward0>), avg_accuracy=63.7]                       


Epoch [19/20], Loss: 0.0086
Epoch 19: Accuracy = 99.54%
Accuracy of the network on the 5000 validation images: 99.01666666666667 %
Epoch 20 ================


844it [01:14, 11.26it/s, avg_loss=tensor(0.0156, device='mps:0', grad_fn=<DivBackward0>), avg_accuracy=63.7]                       


Epoch [20/20], Loss: 0.0307
Epoch 20: Accuracy = 99.55%
Accuracy of the network on the 5000 validation images: 98.91666666666667 %


AttributeError: 'list' object has no attribute 'data'

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs[-1].data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

In [3]:
torch.save(model.state_dict(), "model/Model for Inference.pth")

NameError: name 'torch' is not defined